In [191]:
"""code 
@ https://github.com/Audhil/tensorflow_cookbook/blob/master/02_TensorFlow_Way/03_Working_with_Multiple_Layers/03_multiple_layers.ipynb"""
import tensorflow as tf
import numpy as np
import os

In [192]:
"""Here we will create a small image of size 4x4 pixels and propagate it through multiple layers."""

x_shape = [1, 4, 4, 1]
# random image
x_val = np.random.uniform(size=x_shape)

# data place holder
x_data = tf.placeholder(tf.float32, shape=x_shape)

In [193]:
"""Layer 1 : Convolution"""
# creating 2 x 2 moving window

# Create a layer that takes a spatial moving window average
# Our window will be 2x2 with a stride of 2 for height and width
# The filter value will be 0.25 because we want the average of the 2x2 window
my_filter = tf.constant(.25, shape=[2, 2, 1, 1])
my_strides = [1., 2., 2., 1.]
mov_avg_layer = tf.nn.conv2d(x_data, my_filter, my_strides, padding='SAME', name='moving_window')


In [194]:
"""Layer 2 : Custom"""
"""Our second layer will be a custom layer. 
Given an input, x, this layer flattens out x and computes sigmoid(Ax+b). 
Here, A and b will be predetermined constants."""
# Define a custom layer which will be sigmoid(Ax+b) where
# x is a 2x2 matrix and A and b are 2x2 matrices


def custom_layer(input_tensor):
    input_tensor = tf.squeeze(input_tensor)
    a = tf.constant([[1., 2.], [-3., 4.]])
    b = tf.constant(2., shape=[2, 2])
    return tf.add(tf.matmul(input_tensor, a), b)


# adding custom layer to graph
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_layer)


In [195]:
"""Run the output"""
with tf.Session() as sess:
    # The output should be an array that is 2x2, but size (1,2,2,1)
    output = sess.run(mov_avg_layer, feed_dict={x_data: x_val})
    print('output : ', output)
    print('output.shape : ', output.shape)


output :  [[[[ 0.60947818]
   [ 0.37330738]]

  [[ 0.54317284]
   [ 0.58992469]]]]
output.shape :  (1, 2, 2, 1)


In [196]:
# After custom operation, size is now 2x2 (squeezed out size 1 dims), see below:
with tf.Session() as sess:
    custom_output = sess.run(custom_layer1, feed_dict={x_data: x_val})
    print('custom_output : ', custom_output)
    print('custom_output.shape : ', custom_output.shape)


custom_output :  [[ 1.48955607  4.71218586]
 [ 0.77339876  5.44604445]]
custom_output.shape :  (2, 2)


In [197]:
merged = tf.summary.merge_all(key='summaries')

if not os.path.exists('multi_layer_logs/'):
    os.makedirs('multi_layer_logs/')

my_writer = tf.summary.FileWriter('multi_layer_logs/', sess.graph)
